In [1]:
import pandas as pd

In [2]:
retail = pd.read_csv('data.csv.zip', encoding='ISO-8859-1', compression='zip')

Проверим данные

In [3]:
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [4]:
retail_columns = retail.columns

In [5]:
for i in retail_columns:
    print(i)

InvoiceNo
StockCode
Description
Quantity
InvoiceDate
UnitPrice
CustomerID
Country


Проверим, встречаются ли в данных повторяющиеся наблюдения

In [55]:
retail.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
541904    False
541905    False
541906    False
541907    False
541908    False
Length: 541909, dtype: bool

In [7]:
# Удаление дупликатов
retail.drop_duplicates(inplace=True)

In [8]:
retail.shape[0]

536641

Данные содержат в себе записи как и об успешных транзакциях, так и об отмененных. 
Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 
Узнаем, сколько всего заказов отменили пользователи

In [59]:
# Воспользуемся методом для строк в колонке InvoiceNo, что бы узнать какие строки в этой колонке начинались с  буквы "С"
retail.InvoiceNo.str.startswith('C').sum()

9251

 Теперь отфильтруем данные и оставим в retail только те заказы, где Quantity > 0


In [9]:
retail.query('Quantity > 0').shape[0]


526054

Посчитаем число заказов для каждого пользователя (CustomerID) из Германии (Germany). 
Оставим только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль.
(Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки)


In [10]:
german = retail.query('Country == "Germany"').groupby('CustomerID', as_index=False) \
               .agg({'InvoiceNo': pd.Series.nunique}).rename(columns={'InvoiceNo':'orders_numbers'})

In [11]:
percentile_80 = german.orders_numbers.quantile(q=0.8)

In [12]:
percentile_80

9.0

In [87]:
# Обрашение к переменной через @
#germany_top = german.query('orders_numbers > @percentile_80').index


In [89]:
# или если поставить as_index=False можно просто поставить имя колонки в конце, так как это уже не индекс 
germany_top = german.query('orders_numbers > @percentile_80').CustomerID
germany_top

3     12471.0
4     12472.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
37    12569.0
44    12600.0
52    12619.0
53    12621.0
57    12626.0
65    12647.0
70    12662.0
78    12705.0
79    12708.0
80    12709.0
82    12712.0
84    12720.0
Name: CustomerID, dtype: float64

Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруем наблюдения 
и оставим в данных записи только по интересующим нас юзерам

In [92]:
top_retail_germany = retail.query('CustomerID in @germany_top')
top_retail_germany

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,12/9/2011 11:59,4.95,12662.0,Germany
541727,581570,23077,DOUGHNUT LIP GLOSS,20,12/9/2011 11:59,1.25,12662.0,Germany
541728,581570,20750,RED RETROSPOT MINI CASES,2,12/9/2011 11:59,7.95,12662.0,Germany
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,12/9/2011 11:59,4.95,12662.0,Germany


Сгруппируем top_retail_germany по коду товара (StockCode).
Посмотрим, акой из продуктов добавляли в корзину чаще всего, кроме POST
(одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity)

In [101]:
top_retail_germany.query('StockCode != "POST"').groupby('StockCode').agg({'Quantity': 'count'}).sort_values('Quantity')

,Quantity
StockCode,
72817,1
22128,1
22124,1
22659,1
22119,1
...,...
22077,34
22467,37
22328,45


In [102]:
# или можно просто взять колонку и посчитать сколько строк у одного заказа
top_retail_germany.StockCode.value_counts()

POST     219
22326     62
22423     55
22328     45
22467     37
        ... 
22970      1
21944      1
22902      1
21935      1
23562      1
Name: StockCode, Length: 1168, dtype: int64

Вернемся к анализу полного датасета retail. Создадим колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice

In [103]:
retail['Revenue'] = retail.Quantity * retail.UnitPrice

In [104]:
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60


In [106]:
# Удобнее будет создавать колонки можно методом assign, так как нам надо создать несколько колонок
retail.assign(Revenue = retail.Quantity * retail.UnitPrice)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60


Для каждой транзакции (InvoiceNo), посчитаем финальную сумму заказа.
Выберем топ-5 (InvoiceNo) по сумме заказа

In [114]:
sorted_revenue = retail.groupby('InvoiceNo', as_index=False) \
                       .agg({'Revenue': 'sum'}) \
                       .sort_values('Revenue', ascending=False) \
                       .head(5)
sorted_revenue

,InvoiceNo,Revenue
22025,581483,168469.60
2303,541431,77183.60
18776,574941,52940.94
19468,576365,50653.91
9741,556444,38970.00


In [116]:
sorted_revenue.InvoiceNo.str.cat(sep=', ')

'581483, 541431, 574941, 576365, 556444'